In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import sys, os
from torch_geometric.nn.pool.consecutive import consecutive_cluster

file_path = os.path.dirname(os.path.abspath(''))
sys.path.append(file_path)

from superpoint_transformer.utils import tensor_idx, is_dense, has_duplicates
from superpoint_transformer.data import Data, Cluster, NAG

In [ ]:
num_nodes = 10
num_clusters = 3

x = torch.rand(num_nodes, 2)

edge_index = torch.stack([torch.arange(num_nodes), torch.arange(num_nodes).roll(1)])

# super_index = torch.randint(num_clusters, (num_nodes,))
super_index = torch.LongTensor([2, 0, 0, 1, 2, 0, 1, 1, 0, 1])

d = Data(x=x, super_index=super_index, edge_index=edge_index).cuda()
print(d)

b = PyGBatch.from_data_list([d, d])
print(b)

print()
print(d.super_index)
print(b.super_index)
print(b.to_data_list()[0].super_index)

idx = torch.LongTensor([5, 1, 0]).to(d.device)
print()
print(idx)
print(d.select(idx)[0])
print(d.select(idx)[0].super_index)
print()
print(idx)
print(d.select(idx, update_sub=False)[0])
print(d.select(idx, update_sub=False)[0].super_index)
print()
print(idx)
print(d.select(idx, update_super=False)[0])
print(d.select(idx, update_super=False)[0].super_index)

In [ ]:
device = d.device

# Update the edge indices. To do so, create a tensor of indices
# reindex so that the desired output can be computed with simple
# indexation reindex[edge_index]. This avoids using map() or
# numpy.vectorize alternatives.
reindex = torch.full((d.num_nodes,), -1, dtype=torch.int64, device=device)
reindex = reindex.scatter_(0, idx, torch.arange(idx.shape[0], device=device))

print(idx)
print(edge_index)
print(reindex[edge_index])

edge_index = reindex[edge_index]

# Remove the obsolete edges (ie those involving a '-1' index)
valid_edges = torch.where((edge_index != -1).all(dim=0))[0]
edge_index = edge_index[valid_edges]
print(edge_index)

In [ ]:
from superpoint_transformer.data import Data, Cluster, NAG

import socket
HOST = socket.gethostname()
if HOST == 'DEL2001W017':
    DATA_ROOT = '/media/drobert-admin/DATA2/datasets'
elif HOST == 'HP-2010S002':
    DATA_ROOT = '/var/data/drobert/datasets'
elif HOST == '9c81b1a54ad8':
    DATA_ROOT = '/raid/dataset/pointcloud/data'
else:
    raise NotImplementedError(f"Unknown host '{HOST}', cannot set DATA_ROOT")
    
# nag = torch.load(os.path.join(DATA_ROOT, 'kitti360/shared/temp', 'nag.pt'))
nag = torch.load(os.path.join(DATA_ROOT, 'kitti360/shared/temp', 'nag_20cm.pt'))
# data, data_c = torch.load(os.path.join(DATA_ROOT, 'kitti360/shared/temp', 'preliminaries.pt'))

In [ ]:
print(f'input nag: {nag}')
print(f'input 0: num_nodes={nag[0].num_nodes}, keys={nag[0].keys}')
print(f'input 1: num_nodes={nag[1].num_nodes}, keys={nag[1].keys}')
print()

nag_small = nag.select(1, [10, 20, 100, 500, 1000])

print(f'output nag: {nag_small}')
print(f'output 0: num_nodes={nag_small[0].num_nodes}, keys={nag_small[0].keys}')
print(f'output 1: num_nodes={nag_small[1].num_nodes}, keys={nag_small[1].keys}')

In [ ]:
from superpoint_transformer.data import Cluster

num_clusters = 3
num_points = 50
pointers = torch.randint(num_clusters, (num_points,))
points = torch.arange(num_points)
c = Cluster(pointers, points, dense=True)